## Tasks
Assigned to Sierra Stevenson, JSIP intern under Dr. Jonathan Jiang and Dr. Longtao Wu

List of Tasks<p>
[DONE]&nbsp;&nbsp;&nbsp;Task 1 — UM moisture at 100 hPa map<br>
[DONE]&nbsp;&nbsp;&nbsp;Task 2 — MLS moisture at 100 hPa map<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Task 3 — UM/MLS/AIRS vert. profiles moisture vs temp/pressure<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Task 4 — UM/MLS PDFs (moisture)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Task 5 — UM/GPM_MERGIR PDFs (brightness temp)

for logging in:
* username = smsteven
* password = 2bhzvjM5YfhV.

## Code
Setup

In [ ]:
# NASA Earthdata authorization
from subprocess import Popen
from getpass import getpass
import platform
import os
import shutil

# Data analysis
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Access GPM Data Archive

In [2]:
urs = 'urs.earthdata.nasa.gov'    # Earthdata URL to call for authentication
prompts = ['Enter NASA Earthdata Login Username \n(or create an account at urs.earthdata.nasa.gov): ',
           'Enter NASA Earthdata Login Password: ']

homeDir = os.path.expanduser("~") + os.sep

with open(homeDir + '.netrc', 'w') as file:
    file.write('machine {} login {} password {}'.format(urs, getpass(prompt=prompts[0]), getpass(prompt=prompts[1])))
    file.close()
with open(homeDir + '.urs_cookies', 'w') as file:
    file.write('')
    file.close()
with open(homeDir + '.dodsrc', 'w') as file:
    file.write('HTTP.COOKIEJAR={}.urs_cookies\n'.format(homeDir))
    file.write('HTTP.NETRC={}.netrc'.format(homeDir))
    file.close()

print('Saved .netrc, .urs_cookies, and .dodsrc to:', homeDir)

# Set appropriate permissions for Linux/macOS
if platform.system() != "Windows":
    Popen('chmod og-rw ~/.netrc', shell=True)
else:
    # Copy dodsrc to working directory in Windows  
    shutil.copy2(homeDir + '.dodsrc', os.getcwd())
    print('Copied .dodsrc to:', os.getcwd())

Saved .netrc, .urs_cookies, and .dodsrc to: /users/smsteven/


In [3]:
dataset_url = 'https://disc2.gesdisc.eosdis.nasa.gov/opendap/MERGED_IR/GPM_MERGIR.1/2016/224/merg_2016081100_4km-pixel.nc4'
ds = xr.open_dataset(dataset_url)
ds

<xarray.Dataset>
Dimensions:  (lon: 9896, lat: 3298, time: 2)
Coordinates:
  * lon      (lon) float32 -180.0 -179.9 -179.9 -179.9 ... 179.9 179.9 180.0
  * lat      (lat) float32 -59.98 -59.95 -59.91 -59.87 ... 59.91 59.95 59.98
  * time     (time) datetime64[ns] 2016-08-11 2016-08-11T00:30:00.000013312
Data variables:
    Tb       (time, lat, lon) float32 ...
Attributes:
    BeginDate:       2016-08-11
    BeginTime:       00:00:00.000Z
    EndDate:         2016-08-11
    EndTime:         00:59:59.999Z
    FileHeader:      StartGranuleDateTime=2016-08-11T00:00:00.000Z;\nStopGran...
    InputPointer:    merg_2016081100_4km-pixel
    title:           NCEP/CPC 4km Global (60N - 60S) IR Dataset
    ProductionTime:  2017-11-22T09:04:37.000Z

Create list of URLS

In [4]:
# Function to create lists of numbers that are all two digit strings (ie '00', '01', '02', ...)
def MakeTwoDigits(arr, num_range):
    for num in num_range:
        if num < 10:
            arr.append("0"+f"{num}")
        else:
            arr.append(f"{num}")

# Create empty arrays
month_vals, yday_range, mday_range, hour_range = [], [], [], []

# Fill arrays and modify as needed
MakeTwoDigits(month_vals, np.int_(np.append(np.ones(21)*8, np.ones(8)*9)))      # Month
MakeTwoDigits(yday_range, np.arange(224,252+1))     # Day of the year
MakeTwoDigits(mday_range, np.arange(1,31+1))        # Day of the month
mday_range = mday_range[10:] + mday_range[0:8]      # Day of the month: Reorder to start from the 11 and end at 08 of the next month
MakeTwoDigits(hour_range, np.arange(0,23+1))        # Hour
day_range = list(zip(yday_range, mday_range, month_vals))   # Map day and month arrays together

# Create list to store .nc files
GPM_data_olr_list = []

# Iterate through OPeNDAP URLs and store each dataset in list
for day in day_range:
    for hour in hour_range:
        GPM_data_olr_list.append(f"https://disc2.gesdisc.eosdis.nasa.gov/opendap/MERGED_IR/GPM_MERGIR.1/2016/{day[0]}/merg_2016{day[2]}{day[1]}{hour}_4km-pixel.nc4")

GPM_data_olr_list

['https://disc2.gesdisc.eosdis.nasa.gov/opendap/MERGED_IR/GPM_MERGIR.1/2016/224/merg_2016081100_4km-pixel.nc4',
 'https://disc2.gesdisc.eosdis.nasa.gov/opendap/MERGED_IR/GPM_MERGIR.1/2016/224/merg_2016081101_4km-pixel.nc4',
 'https://disc2.gesdisc.eosdis.nasa.gov/opendap/MERGED_IR/GPM_MERGIR.1/2016/224/merg_2016081102_4km-pixel.nc4',
 'https://disc2.gesdisc.eosdis.nasa.gov/opendap/MERGED_IR/GPM_MERGIR.1/2016/224/merg_2016081103_4km-pixel.nc4',
 'https://disc2.gesdisc.eosdis.nasa.gov/opendap/MERGED_IR/GPM_MERGIR.1/2016/224/merg_2016081104_4km-pixel.nc4',
 'https://disc2.gesdisc.eosdis.nasa.gov/opendap/MERGED_IR/GPM_MERGIR.1/2016/224/merg_2016081105_4km-pixel.nc4',
 'https://disc2.gesdisc.eosdis.nasa.gov/opendap/MERGED_IR/GPM_MERGIR.1/2016/224/merg_2016081106_4km-pixel.nc4',
 'https://disc2.gesdisc.eosdis.nasa.gov/opendap/MERGED_IR/GPM_MERGIR.1/2016/224/merg_2016081107_4km-pixel.nc4',
 'https://disc2.gesdisc.eosdis.nasa.gov/opendap/MERGED_IR/GPM_MERGIR.1/2016/224/merg_2016081108_4km-pixe

Read in all data

In [5]:
print("Now uploading data from NASA Earthdata... (Approximately 22 minutes)")
GPM_data = xr.open_mfdataset(GPM_data_olr_list)
print("Upload complete!")
GPM_data

Now uploading data from NASA Earthdata... (Approximately 13 minutes)
Upload complete!


<xarray.Dataset>
Dimensions:  (lon: 9896, lat: 3298, time: 1392)
Coordinates:
  * lon      (lon) float32 -180.0 -179.9 -179.9 -179.9 ... 179.9 179.9 180.0
  * lat      (lat) float32 -59.98 -59.95 -59.91 -59.87 ... 59.91 59.95 59.98
  * time     (time) datetime64[ns] 2016-08-11 ... 2016-09-08T23:30:00.000026880
Data variables:
    Tb       (time, lat, lon) float32 dask.array<chunksize=(2, 3298, 9896), meta=np.ndarray>
Attributes:
    BeginDate:       2016-08-11
    BeginTime:       00:00:00.000Z
    EndDate:         2016-08-11
    EndTime:         00:59:59.999Z
    FileHeader:      StartGranuleDateTime=2016-08-11T00:00:00.000Z;\nStopGran...
    InputPointer:    merg_2016081100_4km-pixel
    title:           NCEP/CPC 4km Global (60N - 60S) IR Dataset
    ProductionTime:  2017-11-22T09:04:37.000Z

In [11]:
GPM_OLR = GPM_data['Tb']

### Figures

In [ ]:
# All
date_range = "08/11/2016 — 09/08/2016"

# UM
UM_name = "Unified Model"

# MLS
MLS_dates = "08/2004 — 08/2022"
MLS_name = "MLS"

# GPM_MERGIR
GPM_name = "GPM_MERGIR"

In [ ]:
def CreatePDF(arr, prs_hPa="100", data_label=f"Moisture at {prs_hPa} hPa", source=MLS_name, date_range=date_range, custom_units=None):
    # Plot title
    plt.title(f"{source} Data: Probability Density of {data_label} for {date_range}")

    # Unit labels
    if data_label == f"Moisture at {prs_hPa} hPa":
        units = "ppmv"
    elif data_label == "Brightness Temperature":
        units = "K"
    else:
        units = custom_units

    # x and y labels
    plt.xlabel(data_label + f" ({units})")
    plt.ylabel("Density")

    # Plot PDF
    sns.kdeplot(arr)
    plt.show()

Task 4: UM, MLS moisture PDF at 100hPa (hourly data)

In [ ]:
CreatePDF(UM_h2o100_hrly, source=UM_name)
CreatePDF(MLS_h2o100_hrly)

Task 5: UM, GPM_MERGIR brightness temperature PDF (hourly data)

In [7]:
CreatePDF(UM_OLR, data_label="Brightness Temperature", source=UM_name)
CreatePDF(GPM_OLR, data_label="Brightness Temperature", source=GPM_name)

NameError: name 'CreatePDF' is not defined